In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
import sys
import os
from os.path import dirname

UTILS_PATH=os.environ.get('REPO') + "/src/overpass"
sys.path.append(dirname(UTILS_PATH))

UTILS_PATH=os.environ.get('REPO') + "/src/places"
sys.path.append(dirname(UTILS_PATH))

In [3]:
import places

In [4]:
import overpass

In [5]:
import pandas as pd
import numpy as np
def generate_locations_grid(coordinates, grid=0.1):
    x_axis = np.arange(coordinates['lat']['lower'], coordinates['lat']['upper'], grid)
    y_axis = np.arange(coordinates['lon']['lower'], coordinates['lon']['upper'], grid)
    
    results = []
    for i in x_axis:
        for j in y_axis:
            results.append((i, j))
    
    df = pd.DataFrame(results)
    df.columns = ['latitude', 'longitude']
    return df

AMENITIES = ["university", "shopping_mall", "school", "secondary_school", "police", "park", 
         "local_government_office", "library", "hospital", "church", "airport", "amusement_park",
            "bank", "restaurant"]

def build_interest_points_places(df, amenities=AMENITIES):
    results = []
    
    for location_id in df.index:
        location = df.loc[location_id]
        for amenity in amenities:
            results.append(
                places.query_google_nearby(location, amenity)
            )
        
    final = pd.concat(results)
    final.index = list(range(final.shape[0]))
    final.reindex()
    return final



In [10]:
DATE = "2020-10-16"

OSM_TEMPLATE = os.environ.get('REPO') + f"/data/OSM/{DATE}/puntos_interes_{{}}.parquet"
OSM_INPUT_TEMPLATE = os.environ.get('REPO') +  f"/data/OSM/{DATE}/{{}}.geojson"

PLACES_TEMPLATE = os.environ.get('REPO') + f"/data/PLACES/{DATE}/puntos_interes_{{}}.parquet"

In [11]:
from os.path import isfile

def prepare_places_df(locations_grid, filename):
    interest_points = build_interest_points_places(locations_grid)
    
    interest_points = interest_points.loc[:, ["i_lat", "i_lon", "amenity", "name"]]
    interest_points.columns = ["lat", "lon", "amenity", "name"]
    interest_points = interest_points.drop_duplicates(["lat", "lon", "amenity", "name"])
    
    interest_points.to_parquet(filename)
    
    return interest_points

def load_places_results(coordinates, filename, grid=0.01):
    locations_grid = generate_locations_grid(coordinates, grid=grid)
    
    if isfile(filename):
        return pd.read_parquet(filename)
    
    return prepare_places_df(coordinates, filename)

def load_osm_results(filename, osm_filename):    
    if isfile(filename):
        return pd.read_parquet(filename)
    
    osm_df = overpass.read_nodes_overpass_api(osm_filename)
    osm_df.to_parquet(filename)

    return osm_df

## Fusagasuga

In [12]:
fusagasuga_coordinates = {
    'lat': {
        'lower': 4.323127,
        'upper': 4.356505
    },
    'lon':{
        'lower': -74.419864,
        'upper': -74.349483
    }
}

### OSM

Los resultados se descargan en geojson usando el API de overpass

> curl 'https://overpass-api.de/api/interpreter' -H 'Connection: keep-alive' -H 'Accept: */*' -H 'Origin: https://overpass-turbo.eu' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36' -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' -H 'Sec-Fetch-Site: cross-site' -H 'Sec-Fetch-Mode: cors' -H 'Referer: https://overpass-turbo.eu/' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: es,es-419;q=0.9' --data 'data=%2F*%0AThis+has+been+generated+by+the+overpass-turbo+wizard.%0AThe+original+search+was%3A%0A%E2%80%9Camenity%3Dbus_station%E2%80%9D%0A*%2F%0A%5Bout%3Ajson%5D%5Btimeout%3A60%5D%3B%0A%2F%2F+gather+results%0A(%0A++node(4.3114066719897925%2C-74.40919876098633%2C4.359762236889401%2C-74.3488597869873)%3B%0A)%3B%0A%2F%2F+print+results%0Aout+body%3B%0A%3E%3B%0Aout+skel+qt%3B' --compressed  >> fusagasuga.geojson

In [13]:
fusgagasuga_osm_df = load_osm_results(OSM_TEMPLATE.format("fusagasuga"), OSM_INPUT_TEMPLATE.format("fusagasuga"))
fusgagasuga_osm_df.head()

,lat,lon,name,amenity
0,4.3265082,-74.3962481,None,fuel
1,4.3169441,-74.3941878,Fritanga Blanca,restaurant
2,4.3424018,-74.3634092,None,bank
3,4.3319107,-74.3771614,Brio Los Arboles,fuel
4,4.3328698,-74.3702650,None,parking


In [31]:
fusgagasuga_osm_df.shape

(105, 4)

In [33]:
fusgagasuga_osm_df["amenity"].value_counts()

restaurant            14
parking               10
bar                    9
bank                   9
school                 8
fuel                   7
motorcycle_parking     6
fast_food              5
place_of_worship       5
pharmacy               4
community_centre       3
courthouse             3
university             2
townhall               2
hospital               2
waste_disposal         2
post_office            2
cafe                   2
police                 1
childcare              1
car_rental             1
clinic                 1
kindergarten           1
atm                    1
cinema                 1
bus_station            1
public_bookcase        1
library                1
Name: amenity, dtype: int64

### Places

In [11]:
fusagasuga_interest_points = load_places_results(fusagasuga_coordinates, PLACES_TEMPLATE.format("fusagasuga"))
fusagasuga_interest_points.head()

,lat,lon,amenity,name
0,4.327576700000000,-74.38706719999999,university,Alexis León Personal Trainer
1,4.340612600000001,-74.37554469999999,university,CAD Centro Académico Deportivo UCundinamarca
2,4.275842700000000,-74.41282420000000,university,UNAD
3,4.347225000000000,-74.37786919999999,university,UNAD
4,4.337345999999999,-74.37200290000000,university,Auditorio UDEC


In [32]:
fusagasuga_interest_points.shape

(958, 4)

In [34]:
fusagasuga_interest_points["amenity"].value_counts()

restaurant                 429
school                     183
bank                        78
church                      70
shopping_mall               62
park                        59
hospital                    25
university                  18
local_government_office     11
police                       9
library                      7
amusement_park               4
secondary_school             3
Name: amenity, dtype: int64

## Manizales

In [14]:
manizales_coordinates = {
    'lat': {
        'lower': 5.017395,
        'upper': 5.119305
    },
    'lon':{
        'lower': -75.556431,
        'upper': -75.410519
    }
}

### OSM

Los resultados se descargan en geojson usando el API de overpass

> curl 'https://overpass-api.de/api/interpreter' -H 'Connection: keep-alive' -H 'Accept: */*' -H 'Origin: https://overpass-turbo.eu' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36' -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' -H 'Sec-Fetch-Site: cross-site' -H 'Sec-Fetch-Mode: cors' -H 'Referer: https://overpass-turbo.eu/' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: es,es-419;q=0.9' --data 'data=%2F*%0AThis+has+been+generated+by+the+overpass-turbo+wizard.%0AThe+original+search+was%3A%0A%E2%80%9Camenity%3Dbus_station%E2%80%9D%0A*%2F%0A%5Bout%3Ajson%5D%5Btimeout%3A60%5D%3B%0A%2F%2F+gather+results%0A(%0A++node(5.013825705113532%2C-75.55606842041016%2C5.1104360769930555%2C-75.43539047241211)%3B%0A)%3B%0A%2F%2F+print+results%0Aout+body%3B%0A%3E%3B%0Aout+skel+qt%3B' --compressed >> manizales.geojson

In [15]:
manizales_osm_df = load_osm_results(OSM_TEMPLATE.format("manizales"), OSM_INPUT_TEMPLATE.format("manizales"))
manizales_osm_df.head()

,lat,lon,name,amenity
0,5.0644216,-75.4951894,CAI La Leonora,police
1,5.0646280,-75.4974278,101 Perros,fast_food
2,5.0613149,-75.4904344,Universidad de Caldas - Sede Palogrande,university
3,5.0716575,-75.5171473,None,marketplace
4,5.0668861,-75.5042989,Universidad Autónoma de Manizales,university


In [16]:
manizales_osm_df.shape

(693, 4)

### Places

In [17]:
manizales_interest_points = load_places_results(manizales_coordinates, PLACES_TEMPLATE.format("manizales"))
manizales_interest_points.head()

,lat,lon,amenity,name
0,5.045667899999999,-75.52899150000000,university,Universidad Antonio Nariño Manizales
1,5.045667899999999,-75.52899150000000,university,UAN Universidad Antonio Nariño
2,5.001526000000000,-75.59880199999999,university,Mànizales
3,4.985795800000000,-75.60692829999999,university,CUN - Corporación Unificada Nacional de Educac...
4,4.983538500000001,-75.60562440000000,university,JUZGADO PRIMERO PROMISCUO MUNICIPAL DE CHINCHINÁ


In [18]:
manizales_interest_points.shape

(2588, 4)

## Villavicencio

In [22]:
villavicencio_coordinates = {
    'lat': {
        'lower': 4.112948,
        'upper': 4.163473
    },
    'lon':{
        'lower': -73.658594,
        'upper': -73.588900
    }
}

### OSM

Los resultados se descargan en geojson usando el API de overpass

> curl 'https://overpass-api.de/api/interpreter' -H 'Connection: keep-alive' -H 'Accept: */*' -H 'Origin: https://overpass-turbo.eu' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36' -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' -H 'Sec-Fetch-Site: cross-site' -H 'Sec-Fetch-Mode: cors' -H 'Referer: https://overpass-turbo.eu/' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: es,es-419;q=0.9' --data 'data=%2F*%0AThis+has+been+generated+by+the+overpass-turbo+wizard.%0AThe+original+search+was%3A%0A%E2%80%9Camenity%3Dbus_station%E2%80%9D%0A*%2F%0A%5Bout%3Ajson%5D%5Btimeout%3A60%5D%3B%0A%2F%2F+gather+results%0A(%0A++node(4.030028615876668%2C-73.73886108398438%2C4.223502948820042%2C-73.49750518798828)%3B%0A)%3B%0A%2F%2F+print+results%0Aout+body%3B%0A%3E%3B%0Aout+skel+qt%3B' --compressed >> villavicencio.geojson

In [23]:
villavicencio_osm_df = load_osm_results(OSM_TEMPLATE.format("villavicencio"), OSM_INPUT_TEMPLATE.format("villavicencio"))
villavicencio_osm_df.head()

,lat,lon,name,amenity
0,4.1447752,-73.6350492,None,fuel
1,4.1440736,-73.6347972,None,place_of_worship
2,4.1224286,-73.6120109,None,fuel
3,4.1109126,-73.6539156,Los Capachos,nightclub
4,4.1395622,-73.6283019,"Colegio INEM ""Luis Lopez de Mesa""",school


In [24]:
villavicencio_osm_df.shape

(289, 4)

### Places

In [25]:
villavicencio_interest_points = load_places_results(villavicencio_coordinates, PLACES_TEMPLATE.format("villavicencio"))
villavicencio_interest_points.head()

,lat,lon,amenity,name
0,4.110101499999999,-73.65674820000000,university,Universidad Santo Tomas Villavicencio
1,4.108862300000000,-73.65552040000000,university,St. Thomas University - Campus Loma Linda
2,4.110002699999999,-73.66304810000000,university,Hogar Wannie Pooh
3,4.119999400000000,-73.64458770000000,university,CORPORACIÓN UNIVERSITARIA AUTÓNOMA DE NARIÑO -...
4,4.112418500000000,-73.63440080000001,university,Uniminuto


In [26]:
villavicencio_interest_points.shape

(3317, 4)